This script generates all files needed to recreate the data for the manuscript. ~140 GB of storage is needed for the files and ~40 GB of RAM was needed to run certain cells of code.

In [1]:
# %matplotlib inline
import numpy as np
import fourier_prop_1D as fp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tqdm
from scipy.optimize import minimize 
from scipy.optimize import curve_fit
from functools import partialmethod

# help(fp)

#uncomment to disable tqdm output
#tqdm.tqdm.__init__ = partialmethod(tqdm.tqdm.__init__, disable=True)

#define font sizes used for plotting
SMALLER_SIZE = 12
SMALL_SIZE = 16
MEDIUM_SIZE = 22
BIGGER_SIZE = 30




#the file directory that us used to save the files
FILE_DIR='./'

#load custom colormap used in plotting
cmap = np.load("./colormap.npy")
cmap = mpl.colors.ListedColormap(cmap, name='myColorMap', N=cmap.shape[0])

Initialize the laser parameters and simulation window

In [2]:
#center wavelength
wvl_0 = 800e-7 #cm

#FWHM of the spectrum
delta_wvl =100e-7 #cm

#1/exp(2) input beam radius of the field intensity
w_in = 0.05

#length of the spatial simulation window
Lx = 8. #cm

#number of spatial grid points in simulation
Nx = 2**9 + 1

#calculate the 1/exp(2) radius of the spectral intensity used in the calculations
delta_omega = 2 * np.pi * 2.99792458e10 * delta_wvl / (np.sqrt(2 * np.log(2)) * wvl_0**2) #rad/s

#length of the frequency window
L_omega = 120.63552 * delta_omega #rad/s

#number of frequency grid points in simulation 
N_omega=  2**13 + 1

#focal length of lens
focus = 10 #cm

In [4]:
#define fourier propagation parameters for angle of incidence (AOI): 22.8, 36.3, 52.8 degrees, and the linear approximation
aoi228=fp.FourierProp(L=Lx, N=Nx ,N_omega=N_omega, L_omega=L_omega, chirp_pattern='linear_grating_centered', BAR=20, deltax=0., separation=1.97236328, d=1/(1480*10),    aoi=np.deg2rad(22.8), w_in=w_in, focus = focus, delta_wvl=delta_wvl, wvl_0=wvl_0)
aoi36=fp.FourierProp(L=Lx, N=Nx, N_omega=N_omega, L_omega=L_omega, chirp_pattern='linear_grating_centered', BAR=20, deltax=0., separation=4.45043945, d=1/(1480*10), aoi=np.deg2rad(36.3), w_in=w_in, focus = focus, delta_wvl=delta_wvl, wvl_0=wvl_0)
aoi528=fp.FourierProp(L=Lx, N=Nx, N_omega=N_omega, L_omega=L_omega, chirp_pattern='linear_grating_centered', BAR=20, deltax=0., separation=6.59560547, d=1/(1480*10), aoi=np.deg2rad(52.8), w_in=w_in, focus = focus, delta_wvl=delta_wvl, wvl_0=wvl_0)
aoi_linear=fp.FourierProp(L=Lx, N=Nx, N_omega=N_omega, L_omega=L_omega, chirp_pattern='linear', BAR=20.98034668, deltax=0., separation=4.45043945, d=1/(1480*10), aoi=np.deg2rad(36.3), w_in=w_in, focus = focus, delta_wvl=delta_wvl, wvl_0=wvl_0)

#store field names and fields. Make sure both arrays are in the same order
field_names=['aoi228','aoi36','aoi528','aoi_linear']
fields=[aoi228,aoi36,aoi528,aoi_linear]

Generates files for the propagation of the fields in (x,z) space at various times. Each file generated is ~35 GB. Also generates files that track the peak intensity througout propagation

In [ ]:
import time
for n,field in enumerate(fields):
    FILE_NAME=field_names[n]

    #full length of potentail longtitudinal z-values to propagate through.
    #uses the time array of the simulation and converts to z using z=c*t
    zs_full=field.timeshift*1e15*0.3/1e4
    #use only every 8th term due to memory and storage requirements
    zs=zs_full[::8]
    
    #define arrays for the z_slices of propagation and their local time arrays
    z_img=np.zeros((len(zs),field.N_omega,field.N))
    z_times=np.zeros((len(zs),field.N_omega))

    #construct to field and propagate to focus
    field.construct_eField(plot=False)
    field.fraunhofer()

    #loop through z values and store date in arrays
    for i,z in enumerate(tqdm.tqdm(zs)):
        field.aspw(zs=np.array(z),plot=False)
        z_img[i]=np.abs(field.eFieldxt2[::-1,:])**2
        z_times[i]=field.timeshift*1e15+z*1e4/0.3

    #save to file
    np.save(FILE_DIR+'z_img_'+FILE_NAME, z_img)
    np.save(FILE_DIR+'z_times_'+FILE_NAME, z_times)


    #extract values from fields
    timeshift=field.timeshift #seconds
    x2_max=field.x2.max()*1e4 #um

    #define arrays

    #array to store (x,z) fields that will be plotted
    z_plot_t0=np.zeros((len(z_times),Nx))
    #also do every 8 time slice due to memory and storage
    times=timeshift[::8] #seconds

    #define x and z spatial dimensions from simulation
    x2=np.linspace(-x2_max,x2_max,Nx) #um
    z2=np.linspace(-x2_max,x2_max,len(z_times)) #um
    z_max=np.zeros(len(times)) #um
    x_max=np.zeros(len(times)) #um
    I_max=np.zeros(len(times)) #arb. units

    #loop splices together field data that is in (x,t) space into (x,z) space
    for i,t in enumerate(tqdm.tqdm(times)):
        time_val=np.round(t*1e15,3) #fs

        for j in range(len(z_times)):
            
            if (np.argwhere(np.round(z_times[j],3)==time_val)).size>0:
                idx=np.argwhere(np.round(z_times[j],3)==time_val)[0][0]
                z_plot_t0[j]=np.array(z_img[j,idx,:])

            else:
                z_plot_t0[j]=np.zeros((Nx))
    
        #define where the peak intensity is of the field
        max_arg=np.unravel_index((z_plot_t0.T).argmax(), (z_plot_t0.T).shape) 
        #store location and value of the peak
        z_max[i]=(z2[max_arg[1]])
        x_max[i]=(x2[max_arg[0]])
        I_max[i]=(z_plot_t0.T[max_arg])

    #save to file
    np.save(FILE_DIR+'z_max_'+FILE_NAME+'.npy',z_max)
    np.save(FILE_DIR+'x_max_'+FILE_NAME+'.npy',x_max)
    np.save(FILE_DIR+'I_max_'+FILE_NAME+'.npy',I_max)
    

    #the system I was using was low on memory and so this step was needed to clear the memory before going into another iteration of the loop
    #may not be needed on more powerful systsms
    del z_img
    del z_plot_t0
    time.sleep(15)

Generates the files containing fits to the peak intensity values of the tracked position to highlight overall trend of data. The fits are qualitative and not meant to represent any specific funcitonal form.

In [17]:
#high order polynomial fit
def func(x, a, b, c, d, e, f, g, h, i, j):
    return a + b*x + c*x**2 + d*x**3 + e*x**4 + f*x**5 + g*x**6 + h*x**7 + i*x**8 + j*x**9

#linear fit
def func2(x, a, b):
    return a + b*x 

#multiple lorentian fit
def func_I(x, a, b, c, d, e, f, g,):
    return a + b*np.sqrt(1+x**2/c**2)**-d + e*np.sqrt(1+x**2/f**2)**-g

In [18]:
#define the fit for the AOI 22.8 deg case

FILE_NAME='aoi228'

#load in raw data
x_max=np.load(FILE_DIR+'x_max_'+FILE_NAME+'.npy')
z_max=np.load(FILE_DIR+'z_max_'+FILE_NAME+'.npy')
I_max=np.load(FILE_DIR+'I_max_'+FILE_NAME+'.npy')

#define regions to split the fitting to get better results to highlight the overall trend
z1=[None]*len(z_max[0:40])
x1=[None]*len(x_max[0:40])

z2=z_max[40:985]
x2=x_max[40:985]

z3=[None]*len(z_max[985::])
x3=[None]*len(x_max[985::])

#define the best fit for the maximum transverse x location
popt2, pcov2 = curve_fit(func, z2, x2)


#gather data into single array
x_fit=np.concatenate((x1,func(z2, *popt2),x3))

#define the best fit for the peak intensity
popt, pcov = curve_fit(func_I, z_max, I_max, maxfev=60000)

#store into array
I_fit=func_I(z_max, *popt)

#save to file
np.save(FILE_DIR+'x_max_'+FILE_NAME+'_fit.npy',x_fit)
np.save(FILE_DIR+'I_max_'+FILE_NAME+'_fit.npy',I_fit)

In [ ]:
#define the fit for the AOI 36.3 deg case
#throws a couple warnings

FILE_NAME='aoi36'

#load in raw data
x_max=np.load(FILE_DIR+'x_max_'+FILE_NAME+'.npy')
z_max=np.load(FILE_DIR+'z_max_'+FILE_NAME+'.npy')
I_max=np.load(FILE_DIR+'I_max_'+FILE_NAME+'.npy')

#define regions to split the fitting to get better results to highlight the overall trend
z1=[None]*len(z_max[0:11])
x1=[None]*len(x_max[0:11])

z2=z_max[11:375]
x2=x_max[11:375]

z3=z_max[375:650]
x3=x_max[375:650]

z4=z_max[650:1015]
x4=x_max[650:1015]

z5=[None]*len(z_max[1015::])
x5=[None]*len(x_max[1015::])

#define the best fit for the maximum transverse x location
popt2, pcov2 = curve_fit(func, z2, x2)
popt3, pcov3 = curve_fit(func, z3, x3)
popt4, pcov4 = curve_fit(func, z4, x4)


#gather data into single array
x_fit=np.concatenate((x1,func(z2, *popt2),func(z3, *popt3),func(z4, *popt4),x5))

#define the best fit for the peak intensity
popt, pcov = curve_fit(func_I, z_max, I_max, maxfev=60000)

#store into array
I_fit=func_I(z_max, *popt)

#save to file
np.save(FILE_DIR+'x_max_'+FILE_NAME+'_fit.npy',x_fit)
np.save(FILE_DIR+'I_max_'+FILE_NAME+'_fit.npy',I_fit)

In [57]:
#define the fit for the AOI 52.8 deg case

FILE_NAME='aoi528'

#load in raw data
x_max=np.load(FILE_DIR+'x_max_'+FILE_NAME+'.npy')
z_max=np.load(FILE_DIR+'z_max_'+FILE_NAME+'.npy')
I_max=np.load(FILE_DIR+'I_max_'+FILE_NAME+'.npy')

#define regions to split the fitting to get better results to highlight the overall trend
z1=z_max[0:510]
x1=x_max[0:510]

z2=z_max[510:520]
x2=x_max[510:520]

z3=z_max[520::]
x3=x_max[520::]

#define the best fit for the maximum transverse x location
popt1, pcov1 = curve_fit(func, z1, x1)
popt2, pcov2 = curve_fit(func, z2, x2)
popt3, pcov3 = curve_fit(func, z3, x3)


#gather data into single array
x_fit=np.concatenate((func(z1, *popt1),func(z2, *popt2),func(z3, *popt3)))

#define the best fit for the peak intensity
popt, pcov = curve_fit(func_I, z_max, I_max, maxfev=60000)

#store into array
I_fit=func_I(z_max, *popt)

#save to file
np.save(FILE_DIR+'x_max_'+FILE_NAME+'_fit.npy',x_fit)
np.save(FILE_DIR+'I_max_'+FILE_NAME+'_fit.npy',I_fit)

In [58]:
#define the fit for the AOI 52.8 deg case

FILE_NAME='aoi_linear'

#load in raw data
x_max=np.load(FILE_DIR+'x_max_'+FILE_NAME+'.npy')
z_max=np.load(FILE_DIR+'z_max_'+FILE_NAME+'.npy')
I_max=np.load(FILE_DIR+'I_max_'+FILE_NAME+'.npy')

#define regions to split the fitting to get better results to highlight the overall trend
z1=z_max[0:208]
x1=x_max[0:208]

z2=z_max[208:210]
x2=x_max[208:210]

z3=z_max[210:816]
x3=x_max[210:816]

z4=z_max[816:819]
x4=x_max[816:819]

z5=z_max[819::]
x5=x_max[819::]

#define the best fit for the maximum transverse x location
popt1, pcov1 = curve_fit(func, z1, x1)
popt2, pcov2 = curve_fit(func2, z2, x2)
popt3, pcov3 = curve_fit(func, z3, x3)
popt4, pcov4 = curve_fit(func2, z4, x4)
popt5, pcov5 = curve_fit(func, z5, x5)


#gather data into single array
x_fit=np.concatenate((func(z1, *popt1),func2(z2, *popt2),func(z3, *popt3),func2(z4, *popt4),func(z5, *popt5)))

#define the best fit for the peak intensity
popt, pcov = curve_fit(func_I, z_max, I_max, maxfev=60000)

#store into array
I_fit=func_I(z_max, *popt)

#save to file
np.save(FILE_DIR+'x_max_'+FILE_NAME+'_fit.npy',x_fit)
np.save(FILE_DIR+'I_max_'+FILE_NAME+'_fit.npy',I_fit)